## This notebook accomponies the video here

If you have any questions, please post them in the channel page.

In [ ]:
import numpy as np
np.random.seed(1)

import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, chi2, ncx2, uniform, pareto

import utilityFuncs as uf

## Data Generating Process

Imagine that the Data Generating Process (DGP) is based on a Gaussian GARCH(1, 1):


$$Z_t \sim \mathcal{N}(0,\,\sigma_{t}^{2})\,.$$
$$\sigma_{t} = a_0 Z_{t-1}^{2} + a_1 \sigma_{t-1}^{2} + a_2$$


and the target variable (i.e., variable to be forecasted) $Y_t$ simply follows

$$Y_t = \mu + Z_t^2$$

Let's code the above DGP.

In [ ]:
NUM_DAYS = 183
TEST_INDEX = 153
INITIAL_SIGMA = 1
INITIAL_Z = 0


In [ ]:
z_values, sigma_values, days_simulated = uf.simulate_variable_evolution(NUM_DAYS, INITIAL_SIGMA, INITIAL_Z)

The target variable $Y_t$ is 

$$Y_t = \mu + Z_t^2$$

In [ ]:
MU = 1
y_values = [MU + elem**2 for elem in z_values]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(days_simulated, sigma_values, label='Standard Deviation $\sigma$')
ax.plot(days_simulated, z_values, label='Hidden Variable $Z$')

ax.set_xlabel('Days')
ax.set_ylabel('Values')

ax.legend()
plt.show()



In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.plot(days_simulated, y_values, label='True')


Create a dataframe to save days and actual values

In [ ]:
df_actual_values = pd.DataFrame({'days': days_simulated, 'actual_values':y_values}, columns=['days','actual_values'])
df_actual_values.to_csv('df_actual_values.csv', sep='\t', float_format='%.4f')


In [ ]:
df_actual_values

## Expert Forecasts

### Mean
Let's first study the case where expert submits the conditional mean at each time as the forecast for that time. 


The conditional expectation of $Y_t$ at each time is obtained as follows. 

$$ \mathop{\mathbb{E}(Y_t\mid t)} = \mathop{\mathbb{E}(\mu + Z_t^2\mid t)} = \mu + \mathop{\mathbb{E}(Z_t^2\mid t)} = \mu + Var(Z_t\mid t) + \mathop{\mathbb{E}(Z_t\mid t)^2} = \mu + \sigma_t^2$$

where the first three equalities follow simply from the properties of expectation operator and the last equality follows from the fact that $Z_t \sim \mathcal{N}(0,\,\sigma_{t}^{2})\,$ (see above).


In [ ]:
# Forecasts
expert_forecasts = [MU + elem**2 for elem in sigma_values[TEST_INDEX:]]

In [ ]:
shared_data = y_values[0:TEST_INDEX]
mean_value = sum(shared_data)/len(shared_data)
max_value = max(shared_data)
min_value = min(shared_data)

naive_mean_forecasts = [mean_value] * len(expert_forecasts)
naive_max_forecasts = [max_value] * len(expert_forecasts)
naive_min_forecasts = [min_value] * len(expert_forecasts)

In [ ]:
TEST_DAYS = [day for day in range(TEST_INDEX, NUM_DAYS)]

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(TEST_DAYS, y_values[TEST_INDEX:], label='True')
plt.plot(TEST_DAYS, expert_forecasts, label='Expert')
plt.plot(TEST_DAYS, naive_mean_forecasts, label='Naive Mean')
plt.plot(TEST_DAYS, naive_max_forecasts, label='Naive Max')
plt.plot(TEST_DAYS, naive_min_forecasts, label='Naive Min')

plt.xlabel('Test Days')
plt.ylabel('Forecast Values')
plt.title('Forecast Comparison')

plt.legend()
plt.show()


## First Test Day

Let's focus on what happens during the first test day.

In [ ]:
sigma_values[TEST_INDEX]

In [ ]:
def pdf_expression(mean, std, x):
    "See notebook Transformations.ipynb"
    return (1. / (2. * std * np.sqrt(2. * np.pi * (x - mean)))) * np.exp(-(x - mean) / (2. * std ** 2.))

def Y_pdf(mean, std):
    return lambda x: pdf_expression(mean, std, x)


x = np.array(TEST_DAYS[:1]) - TEST_DAYS[0]

# Define the values of x_1 and x_2
TEST_IND_1 = 0

x_1 = 0

sig_1 = sigma_values[TEST_INDEX + TEST_IND_1]
#chi2_non_centrality_param = MU**2/(2*sig_1**2)

plt.figure(figsize=(12, 4))

# Plot the curve of y_hat = sin(x)
plt.scatter(x, y_values[TEST_INDEX:TEST_INDEX+1], label='True Values')
plt.scatter(x, expert_forecasts[0:1], label='Expert Forecasts')

# Plot the conditional distribution of Y at x_1 and x_2
for xi, sigi, label in [(x_1, sig_1, 'x_1')]:    
    y = np.linspace(1, 7, 1000)    
    plt.fill_betweenx(y, xi, xi + pdf_expression(MU, sig_1, y), alpha=0.5, label=f'Y given {label}')

# Add legend and labels
plt.legend()
plt.xlabel('Test Day')
plt.ylabel('y')
plt.xlim([-0.1, 1])
plt.ylim([.9, 6])
plt.title('First Test Day: Underlying Distribution, True Value and Expert Forecast')

# Show the plot
plt.show()

In [ ]:
#n = 10
n = 100000
y_tildas = np.linspace(MU, MU+4, 100).reshape(-1, 1)


Y_scale = 100
#Y_scale = 10
Y, _ = uf.importance_sampling(Y_pdf(MU, sig_1), uniform(loc=MU, scale=Y_scale), n)



In [ ]:
len(Y [Y == min(Y)])/n

In [ ]:
print(Y.mean(), expert_forecasts[0])


In [ ]:
# Calculate error metrics
MSE_values = np.mean((Y - y_tildas) ** 2, axis=1)
MAE_values = np.mean(np.abs(Y - y_tildas), axis=1)
MAPE_values = np.mean(np.abs((Y - y_tildas) / Y), axis=1)

In [ ]:
np.savetxt('selected_Y.csv', Y, delimiter=',') 
np.savetxt('y_tildas.csv', y_tildas, delimiter=',') 
np.savetxt('MSE_values.csv', MSE_values, delimiter=',') 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))
axes[0].hist(Y)
axes[0].set_title("Histogram of Y")

axes[1].plot(y_tildas, MSE_values)
axes[1].vlines(np.mean(Y), ymin=min(MSE_values), ymax=max(MSE_values), colors="r", label="Mean")
axes[1].vlines(expert_forecasts[0], ymin=min(MSE_values), ymax=max(MSE_values), colors="b", label="expert_forecasts[0]")
axes[1].set_ylabel("Mean Squared Error")
axes[1].set_xlabel(r"$\tilde{y}$")
axes[1].text(np.mean(Y), max(MSE_values), f'mean(Y)', ha='center', va='bottom', color='r', fontsize=10)
axes[1].legend()
fig.suptitle('First Test Day')
